In [1]:
import pandas as pd
import plotly.graph_objs as go

In [2]:
# ✅ Load data using a relative path
df = pd.read_csv("../ProcessDataIntegration/output/Interpolated_Geometry_With_Process_Params.csv")

In [3]:
import plotly.graph_objs as go

# 📊 2. Segmentation by arc_id
precision = 1.0
segmented = []
for lid in sorted(df['layer_id'].unique()):
    layer_df = df[df['layer_id'] == lid].copy()
    layer_df['arc_id'] = (layer_df['X'] / precision).round().astype(int)
    layer_df['arc_id'] = layer_df['arc_id'] - layer_df['arc_id'].min() + 1  # ← starts from 1
    segmented.append(layer_df)

df_all = pd.concat(segmented, ignore_index=True)

# 🧠 3. Hover info
df_all['info'] = (
    "Layer: " + df_all['layer_id'].astype(str) +
    "<br>Arc ID: " + df_all['arc_id'].astype(str) +
    "<br>I: " + df_all['proc_I'].round(2).astype(str) +
    "<br>U: " + df_all['proc_U'].round(2).astype(str) +
    "<br>WFS: " + df_all['proc_WFS'].round(2).astype(str) +
    "<br>Speed: " + df_all['proc_speed'].round(2).astype(str) +
    "<br>X=" + df_all['X'].round(2).astype(str) +
    "<br>Y=" + df_all['Y'].round(2).astype(str) +
    "<br>Z=" + df_all['Z'].round(2).astype(str)
)

# ⚙️ 4. Z-based anomaly detection with neighbors (trimmed)
threshold_z_drop = 1.0
layer_ids = sorted(df_all['layer_id'].unique())
trimmed_anomaly_points = []

for i in range(1, len(layer_ids) - 1):
    lid_prev, lid_curr, lid_next = layer_ids[i - 1], layer_ids[i], layer_ids[i + 1]
    df_prev = df_all[df_all['layer_id'] == lid_prev]
    df_curr = df_all[df_all['layer_id'] == lid_curr]
    df_next = df_all[df_all['layer_id'] == lid_next]

    common_arc_ids = set(df_curr['arc_id']).intersection(df_prev['arc_id'], df_next['arc_id'])

    for arc_id in common_arc_ids:
        curr_arc = df_curr[df_curr['arc_id'] == arc_id].sort_values(by='X')
        prev_arc = df_prev[df_prev['arc_id'] == arc_id].sort_values(by='X')
        next_arc = df_next[df_next['arc_id'] == arc_id].sort_values(by='X')

        def trim(df_arc):
            n = len(df_arc)
            if n < 10:
                return df_arc
            return df_arc.iloc[int(n * 0.05): int(n * 0.95)]

        trimmed_curr = trim(curr_arc)
        z_curr = trimmed_curr['Z'].mean()
        z_prev = trim(prev_arc)['Z'].mean()
        z_next = trim(next_arc)['Z'].mean()

        z_neighbors_avg = (z_prev + z_next) / 2

        if (z_neighbors_avg - z_curr) > threshold_z_drop:
            trimmed_anomaly_points.append(trimmed_curr)

# 📦 5. Combine anomaly points
df_trimmed_anomalies = pd.concat(trimmed_anomaly_points, ignore_index=True)
df_trimmed_anomalies['Anomalies'] = True

# 🔴 6. Mark anomalies in df_all
df_all['Anomalies'] = False
trimmed_keys = set(zip(
    df_trimmed_anomalies['X'].round(5),
    df_trimmed_anomalies['Y'].round(5),
    df_trimmed_anomalies['Z'].round(5)
))
df_all['Anomalies'] = df_all.apply(
    lambda row: (round(row['X'], 5), round(row['Y'], 5), round(row['Z'], 5)) in trimmed_keys,
    axis=1
)

# 🧪 7. Save to CSV (optional)

# 📊 8. Visualization
unique_layers = sorted(df_all['layer_id'].unique())
layer_id_map = {old: new + 1 for new, old in enumerate(unique_layers)}
df_all['layer_id_mapped'] = df_all['layer_id'].map(layer_id_map)
df_trimmed_anomalies['layer_id_mapped'] = df_trimmed_anomalies['layer_id'].map(layer_id_map)
layer_colors = ['black'] * len(unique_layers)

fig = go.Figure()

# ⚫ All points by layer
for lid in sorted(df_all['layer_id_mapped'].unique()):
    layer_df = df_all[df_all['layer_id_mapped'] == lid]
    fig.add_trace(go.Scatter3d(
        x=layer_df['X'], y=layer_df['Y'], z=layer_df['Z'],
        mode='markers',
        name=f"Layer {lid}",
        text=layer_df['info'],
        marker=dict(size=2, color=layer_colors[lid - 1], opacity=0.8),
        hoverinfo='text',
        visible=True
    ))

# 🔴 Trimmed anomalies
for lid in sorted(df_trimmed_anomalies['layer_id_mapped'].unique()):
    trimmed_layer_df = df_trimmed_anomalies[df_trimmed_anomalies['layer_id_mapped'] == lid]
    fig.add_trace(go.Scatter3d(
        x=trimmed_layer_df['X'], y=trimmed_layer_df['Y'], z=trimmed_layer_df['Z'],
        mode='markers',
        name=f"Anomalies (Layer {lid})",
        text=trimmed_layer_df['info'],
        marker=dict(size=3, color='red', opacity=1.0),
        hoverinfo='text',
        visible=True
    ))

# 📐 Plot layout configuration
x_range = [df_all['X'].min(), df_all['X'].max()]
y_range = [df_all['Y'].min(), df_all['Y'].max()]
z_range = [df_all['Z'].min(), df_all['Z'].max()]

axis_length = max(
    x_range[1] - x_range[0],
    y_range[1] - y_range[0],
    z_range[1] - z_range[0]
)

fig.update_layout(
    title="Anomalies",
    scene=dict(
        xaxis=dict(title='X', range=[x_range[0], x_range[0] + axis_length]),
        yaxis=dict(title='Y', range=[y_range[0], y_range[0] + axis_length]),
        zaxis=dict(title='Z', range=[z_range[0], z_range[0] + axis_length]),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.4),
        camera=dict(eye=dict(x=1.25, y=1.25, z=1.25))
    ),
    height=750,
    showlegend=True
)

# fig.show()
pass


In [4]:
import os

# 📁 Path to the output directory next to this notebook
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

# 📄 Path to the HTML file
output_path = os.path.join(output_dir, "3D_DetectAnomalies_layers.html")

# 💾 Save the figure
fig.write_html(output_path)
print(f"✅ Plot saved: {output_path}")


✅ Plot saved: ./output\3D_DetectAnomalies_layers.html


In [5]:
# === 6. Save the result to CSV ===
from pathlib import Path

# Create the output folder (if it doesn't exist yet)
output_dir = Path("output")
output_dir.mkdir(parents=True, exist_ok=True)

# Path to the file
output_path = output_dir / "DetectAnomalies.csv"

# Save the DataFrame
df_all.to_csv(output_path, index=False)

print(f"✅ Saved to: {output_path.resolve()}")

✅ Saved to: C:\Users\kopil\OneDrive\Рабочий стол\Projects\Additive_lab_robots\RaTSiF\data\framework\src\AnomalyDetection\output\DetectAnomalies.csv
